In [1]:
!pip install SPARQLWrapper

  Using cached SPARQLWrapper-2.0.0-py3-none-any.whl.metadata (2.0 kB)
  Using cached rdflib-7.1.1-py3-none-any.whl.metadata (11 kB)
Using cached SPARQLWrapper-2.0.0-py3-none-any.whl (28 kB)
Using cached rdflib-7.1.1-py3-none-any.whl (562 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests

# Set up the SPARQL wrapper
endpoint_url = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(endpoint_url)


In [6]:


def save_gpx(qid:str):
  query = f"""
  #Wielrenners
  #title: Wielrenners
  SELECT DISTINCT ?race ?raceQid ?raceLabel ?startPointLabel ?startPointLongitude ?startPointLatitude  ?endPointLabel ?endPointLongitude ?endPointLatitude WHERE {{
    ?race wdt:P1346|wdt:P2321|wdt:P710 wd:{qid};

          
          
            wdt:P1427 ?startPoint;
            wdt:P1444 ?endPoint.
    ?startPoint wdt:P17 ?startPointCountry ;
                wdt:P625 ?startPointCoords .
    BIND(geof:latitude(?startPointCoords) AS ?startPointLatitude)
      BIND(geof:longitude(?startPointCoords) AS ?startPointLongitude)
    ?endPoint wdt:P17 ?endPointCountry ;
              wdt:P625 ?endPointCoords .
      BIND(geof:latitude(?endPointCoords) AS ?endPointLatitude)
      BIND(geof:longitude(?endPointCoords) AS ?endPointLongitude)
      BIND(STRAFTER(STR(?race), STR(wd:)) AS ?raceQid) 
    
  
    FILTER(?endPoint != ?startPoint && (?endPointCountry IN ( wd:Q31, wd:Q55 ) || ?endPointCountry IN ( wd:Q31, wd:Q55 )) )
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "nl,en,fr,mul". }}
  }}
  """


  # Specify the query and format
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)

  # Execute the query and get the results
  results = sparql.query().convert()

  # Parse and print the results
  for result in results["results"]["bindings"]:
      race = result["raceQid"]["value"]
      body = {"coordinates":[[float(result["startPointLongitude"]["value"]),float(result["startPointLatitude"]["value"])],[float(result["endPointLongitude"]["value"]),float(result["endPointLatitude"]["value"])]]}
      
      print(race)
      headers = {
          'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
          'Authorization': '5b3ce3597851110001cf62489354ef714b2f4f2ca26b24494639a68c',
          'Content-Type': 'application/json; charset=utf-8'
      }
      call = requests.post('https://api.openrouteservice.org/v2/directions/cycling-regular/gpx', json=body, headers=headers)

      print(call.status_code, call.reason)
      with open(f'data/{race}.xml', 'w') as file:
          print(call.text, file=file)
# Q234158
# Q536470
# Q454963
# Q925964
for qid in ["Q925964"]:
   save_gpx(qid)

Q21856733
200 OK
